Part #1 Segmenting and Clustering Assignment

*Data Gathering*

In [3]:
pip install bs4

     |████████████████████████████████| 102kB 18.7MB/s ta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install lxml

     |████████████████████████████████| 5.8MB 25.5MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [7]:
# import necessary libraries for data analysis and processing.
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import urllib3.request
from sklearn.cluster import KMeans
import folium
import os
import requests
import json
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors

In [8]:
# get data from wiki and create soup object.
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(source.text, 'lxml')

In [9]:
#use our soup object to extract wikitable data from html page and store it into a list.
data = []
columns = []
table = soup.find(class_='wikitable')
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    
    #set first row of the list to be its header.
    if (index == 0):
        columns = section
    else:
        data.append(section)

#convert list into pd DataFrame.
ca_df = pd.DataFrame(data = data,columns = columns)
ca_df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


*Data Wrangling*

In [11]:
#new dataframe with Neighbourhoods that are labeled as not assigned.
dfN=ca_df
ca_df1 = dfN.loc[dfN['Neighbourhood']== 'Not assigned']

#replace Not assigned Neighbourhoods column values with Borough column.
for i in ca_df1.index:
    ca_df1.at[i, 'Neighbourhood'] = ca_df1.at[i, 'Borough']

#replace rows in DataFrame.
dfN.loc[ca_df1.index] = ca_df1

#delete rows with Borough that are not assigned.
ca_df = dfN[dfN.Borough != 'Not assigned']

#group by postcode and Borough, keeping all the Neighbourhoods.
ca_df=ca_df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda x: ','.join(x.astype(str))).reset_index()

#remove duplicates.
ca_df = ca_df.drop_duplicates()
ca_df

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie..."
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."


In [12]:
ca_df.shape

(103, 3)

#Assigning Geospatial Location

In [15]:
#loading geographical locations of Toronto postcodes.
Geo_loc = pd.read_csv('https://cocl.us/Geospatial_data', header=0)
Geo_loc.rename(columns={'Postal Code':'Postcode'}, inplace=True)

#assigning postcodes within DataFrame.
df_Glo=pd.merge(ca_df,Geo_loc, on='Postcode')
df_Glo

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739416,-79.588437


#Visualizing Clusters of Toronto Boroughs using Kmeans

In [17]:
toronto_map = folium.Map(location=[43.65, -79.4], zoom_start=12)

X = df_Glo['Latitude']
Y = df_Glo['Longitude']
Z = np.stack((X, Y), axis=1)

kmeans = KMeans(n_clusters=4, random_state=0).fit(Z)

clusters = kmeans.labels_
colors = ['red', 'green', 'blue', 'yellow']
df_Glo['Cluster'] = clusters

for latitude, longitude, borough, cluster in zip(df_Glo['Latitude'], df_Glo['Longitude'], df_Glo['Borough'], df_Glo['Cluster']):
    label = folium.Popup(borough, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color=colors[cluster],
        fill_opacity=0.7).add_to(toronto_map)  

toronto_map

In [20]:
# filter borough names that contain the word Toronto
borough_names = list(df_Glo.Borough.unique())

toronto_borough= []

for x in borough_names:
    if "toronto" in x.lower():
        toronto_borough.append(x)
        
toronto_borough

['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']

In [23]:
# new DataFrame with only boroughs in Toronto
df_Toronto = df_Glo[df_Glo['Borough'].isin(toronto_borough)].reset_index(drop=True)
print(df_Toronto.shape)
df_Toronto.head()

(39, 6)


,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,0
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,0
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,1


**Using API Foursquare to explore neighbourhoods**

In [30]:
# define Foursquare Credentials and Version
CLIENT_ID = 'SQARXQRVULTRTJESUOPUSLUZMQO5Y5SRIQOGQUHEIKQ0CGHB' # your Foursquare ID
CLIENT_SECRET = 'UAQO5PMEJYI2VAM3HKWQDU1YAZHI4TYZXWWLTE55I5AHLXRD' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SQARXQRVULTRTJESUOPUSLUZMQO5Y5SRIQOGQUHEIKQ0CGHB
CLIENT_SECRET:UAQO5PMEJYI2VAM3HKWQDU1YAZHI4TYZXWWLTE55I5AHLXRD


**Identifying top 100 venues within 500m radius**

In [31]:
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(df_Toronto['Latitude'], df_Toronto['Longitude'], df_Toronto['Postcode'], df_Toronto['Borough'], df_Toronto['Neighbourhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [32]:
# convert the venues list into a new DataFrame
df_venues = pd.DataFrame(venues)

# define the column names
df_venues.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(df_venues.shape)
df_venues.head()

(1674, 9)


,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,East Toronto,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,East Toronto,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,Dip 'n Sip,43.678897,-79.297745,Coffee Shop


**Top 100 Venues  Neighborhood**

In [33]:
df_venues.groupby(["PostalCode", "Borough", "Neighborhood"]).count()

,,,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
PostalCode,Borough,Neighborhood,,,,,,
M4E,East Toronto,The Beaches,5,5,5,5,5,5
M4K,East Toronto,"The Danforth West,Riverdale",41,41,41,41,41,41
M4L,East Toronto,"The Beaches West,India Bazaar",19,19,19,19,19,19
M4M,East Toronto,Studio District,40,40,40,40,40,40
M4N,Central Toronto,Lawrence Park,4,4,4,4,4,4
M4P,Central Toronto,Davisville North,9,9,9,9,9,9
M4R,Central Toronto,North Toronto West,20,20,20,20,20,20
M4S,Central Toronto,Davisville,33,33,33,33,33,33
M4T,Central Toronto,"Moore Park,Summerhill East",3,3,3,3,3,3


In [34]:
print('There are {} uniques categories.'.format(len(df_venues['VenueCategory'].unique())))

There are 231 uniques categories.
